# Pointeur

## Adresse mémoire
Quand on définit une variable (que ce soit en C, OCaml ou Python), sa valeur est stockée dans la mémoire RAM de l'ordinateur. La mémoire RAM est décomposée en cases, chaque case occupant 8 bits (1 octet) et possédant une **adresse mémoire**.  
On peut connaître l'adresse mémoire où est stockée une variable avec `&` :

In [1]:
int x = 3;
&x  // affichage de l'adresse mémoire où est stockée la valeur de x

@0x7ffc1e147378

Un **pointeur** est une variable dont la valeur est une adresse mémoire. On utilise `*` dans un type pour définir un pointeur. Par exemple, un `float*` est un pointeur sur un `float`.

In [2]:
int* p = &x;  // p est un pointeur sur un int
p

@0x7ffc1e147378

Il est possible de modifier l'adresse mémoire contenu dans un pointeur :

In [3]:
int y = x + -42;
&y

@0x7ffc1e147378

In [4]:
printf("%p\n", &x);
printf("%p", &y);

0x7f2a95432028
0x7f2a95432038

In [5]:
p = &y;
p

@0x7ffc1e147378

In [6]:
! gcc

gcc: fatal error: no input files
compilation terminated.
